In [1]:
import pandas as pd
import nltk
import torch
import numpy as np
nltk.download('punkt')
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [2]:
!pip install --upgrade datasets
!pip install --upgrade fsspec


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 19.9 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [3]:
from datasets import load_dataset
dataset = load_dataset("cnn_dailymail", "3.0.0",split='train[:5%]')


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [4]:
import pandas as pd
df = pd.DataFrame(dataset)

# 📌 Sentence splitting
def split_sentences(text):
    return sent_tokenize(text)

df['sentences'] = df['article'].apply(split_sentences)

In [5]:
for i, row in df.head(2).iterrows():
    print(f"\n📰 Article {i+1} sentences:")
    for sent in row['sentences']:
        print(f" - {sent}")


📰 Article 1 sentences:
 - LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.
 - Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.
 - "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month.
 - "I don't think I'll be particularly extravagant.
 - "The things I like buying are things that cost about 10 pounds -- books and CDs and DVDs."
 - At 18, Radcliffe will be able to gamble in a casino, buy a drink in a pub or see the horror film "Hostel: Part II," currently six places below his number one movie 

In [6]:
!pip install transformers --quiet
from transformers import BertTokenizer, BertModel
import torch
from tqdm import tqdm

In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
bert_model = BertModel.from_pretrained('bert-base-uncased')
bert_model.eval()
bert_model.to('cuda')


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False

In [8]:
def get_sentence_embedding(sent):
  tokens = tokenizer(sent,return_tensors='pt',truncation=True,max_length=512,padding='max_length')
  tokens = {k: v.to('cuda') for k, v in tokens.items()}
  with torch.no_grad():
    outputs = bert_model(**tokens)
  cls_embedding = outputs.last_hidden_state[:,0,:]
  return cls_embedding.squeeze().cpu().numpy()

sentence_embeddings = []
print("Encoding sentences with BERT...")

Encoding sentences with BERT...


In [9]:
for sentences in tqdm(df['sentences'][:1000]):
  article_embeddings = []
  for sent in sentences:
    emb = get_sentence_embedding(sent)
    article_embeddings.append(emb)
  sentence_embeddings.append(article_embeddings)

100%|██████████| 1000/1000 [22:26<00:00,  1.35s/it]


In [13]:
# 📌 Sanity check function to verify sentence embeddings
def sanity_check(index=0):
    print(f"\n🧪 Sanity Check for Article #{index + 1}")
    print("=" * 50)

    sentences = df['sentences'][index]
    embeddings = sentence_embeddings[index]

    for i, (sent, emb) in enumerate(zip(sentences, embeddings)):
        print(f"📝 Sentence {i+1}: {sent}")
        print(f"🔢 Embedding shape: {np.array(emb).shape}")
        print("-" * 50)

# ✅ Run sanity check on first article
sanity_check(0)




🧪 Sanity Check for Article #1
📝 Sentence 1: LONDON, England (Reuters) -- Harry Potter star Daniel Radcliffe gains access to a reported £20 million ($41.1 million) fortune as he turns 18 on Monday, but he insists the money won't cast a spell on him.
🔢 Embedding shape: (768,)
--------------------------------------------------
📝 Sentence 2: Daniel Radcliffe as Harry Potter in "Harry Potter and the Order of the Phoenix" To the disappointment of gossip columnists around the world, the young actor says he has no plans to fritter his cash away on fast cars, drink and celebrity parties.
🔢 Embedding shape: (768,)
--------------------------------------------------
📝 Sentence 3: "I don't plan to be one of those people who, as soon as they turn 18, suddenly buy themselves a massive sports car collection or something similar," he told an Australian interviewer earlier this month.
🔢 Embedding shape: (768,)
--------------------------------------------------
📝 Sentence 4: "I don't think I'll be parti

In [10]:
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
class SentenceClassifier(nn.Module):
  def __init__(self,input_size=384,hidden_size=128):
    super(SentenceClassifier,self).__init__()
    self.fc = nn.Sequential(
        nn.Linear(input_size,hidden_size),
        nn.ReLU(),
        nn.Linear(hidden_size,1),
        nn.Sigmoid()
    )
  def forward(self, x):
    return self.fc(x)


In [12]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer('all-MiniLM-L6-v2')

 # or any other model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = SentenceClassifier(input_size=384).to(device)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:

from sklearn.metrics.pairwise import cosine_similarity

def generate_labels(article_sents, summary_text, model):
    labels = []

    # Encode the full summary once (768-d vector)
    summary_emb = model.encode([summary_text])  # shape: (1, 768)

    for sent in article_sents:
        sent_emb = model.encode([sent])  # shape: (1, 768)

        # Compute cosine similarity between sentence and summary
        sim = cosine_similarity(sent_emb, summary_emb)[0][0]

        # Binary label based on threshold
        labels.append(1 if sim > 0.7 else 0)

    return labels


In [14]:
print("Training classifier on 1000 articles...")

Training classifier on 1000 articles...


In [80]:
print(embeddings.shape)  # Should be [num_sents, 384]
print(labels.shape)      # Should be [num_sents, 1]


torch.Size([384])
torch.Size([24, 1])


In [15]:
for epoch in range(5):
  total_loss = 0
  for i in range(1000):
    sents = df['sentences'][i]
    summary = df['highlights'][i]
    embeddings = embedding_model.encode(sents, convert_to_tensor=True).to(device)
    labels = torch.tensor(generate_labels(sents,summary,embedding_model),dtype=torch.float32).unsqueeze(1).to(device)
    preds = model(embeddings)
    loss = criterion(preds,labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    total_loss += loss.item()
  print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}")

Epoch 1 - Loss: 381.5341
Epoch 2 - Loss: 185.3392
Epoch 3 - Loss: 176.2050
Epoch 4 - Loss: 171.5351
Epoch 5 - Loss: 167.8379


In [16]:
def generate_extractive_summary(article_sentences, model, classifier, top_k=3):
    # Step 1: Encode all sentences using sentence-transformers
    sentence_embeddings = embedding_model.encode(article_sentences, convert_to_tensor=True)
    device = next(classifier.parameters()).device
    sentence_embeddings = sentence_embeddings.to(device)
    # Step 2: Get prediction scores from classifier
    with torch.no_grad():
        scores = classifier(sentence_embeddings).squeeze()

    # Step 3: Get indices of top-k scores
    top_k_indices = torch.topk(scores, k=min(top_k, len(article_sentences))).indices.tolist()

    # Step 4: Sort selected sentences in original order
    top_k_indices.sort()
    summary = [article_sentences[i] for i in top_k_indices]
    return summary


In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
i = 2
article_sents = df['sentences'][i]
reference_summary = df['highlights'][i]

# Generate predicted summary
predicted_summary = generate_extractive_summary(article_sents, embedding_model, model, top_k=3)

# Display
print(f"\n🔸 Reference Summary:\n{reference_summary}")
print("\n🔹 Predicted Extractive Summary:")
for sent in predicted_summary:
    print(f"- {sent}")



🔸 Reference Summary:
NEW: "I thought I was going to die," driver says .
Man says pickup truck was folded in half; he just has cut on face .
Driver: "I probably had a 30-, 35-foot free fall"
Minnesota bridge collapsed during rush hour Wednesday .

🔹 Predicted Extractive Summary:
- Occasionally, a pickup truck with a medic inside would drive to get an injured person and bring him back up even ground, Hink told CNN.
- Babineau said the rear of his pickup truck was dangling over the edge of a broken-off section of the bridge.
- I literally thought I was dead.


In [18]:
!pip install rouge-score --quiet
from rouge_score import rouge_scorer

def evaluate_rouge(predicted_sents, reference_summary):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
    generated = ' '.join(predicted_sents)
    scores = scorer.score(reference_summary, generated)
    return scores

# Example
scores = evaluate_rouge(predicted_summary, reference_summary)
print("🔍 ROUGE Scores:", scores)


  Preparing metadata (setup.py) ... done
🔍 ROUGE Scores: {'rouge1': Score(precision=0.19230769230769232, recall=0.24390243902439024, fmeasure=0.21505376344086025), 'rougeL': Score(precision=0.11538461538461539, recall=0.14634146341463414, fmeasure=0.12903225806451613)}


In [19]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [20]:
# Save the entire model
torch.save(model.state_dict(), '/content/drive/MyDrive/sentence_classifier.pth')
